In [2]:
import numpy as np
import h5py
import os
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [3]:
with h5py.File('/home/yansheng/kaggle_dog/resnet50_v2_pretrained_Xy.h5', 'r') as f:
    X_train_resnet50_v2 = np.array(f['X_train_resnet50_v2'])
    X_test_resnet50_v2 = np.array(f['X_test_resnet50_v2'])
    y_train = np.array(f['y_train'])
    
print ("X_train Size: ", X_train_resnet50_v2.shape)
print ("X_test Size: ", X_test_resnet50_v2.shape)
print ("y_train Size: ", y_train.shape)

X_train Size:  (10222, 2048)
X_test Size:  (10357, 2048)
y_train Size:  (10222,)


In [4]:
X_train, X_val, y_train, y_val = train_test_split(X_train_resnet50_v2, y_train, test_size=0.2)

# dataset
dataset_train = mx.gluon.data.ArrayDataset(nd.array(X_train), nd.array(y_train))
dataset_val = mx.gluon.data.ArrayDataset(nd.array(X_val), nd.array(y_val))

# data itet
batch_size = 128
data_iter_train = mx.gluon.data.DataLoader(dataset_train, batch_size, shuffle=True)
data_iter_val = mx.gluon.data.DataLoader(dataset_val, batch_size)

In [5]:
ctx = mx.gpu()
def get_net(ctx):

    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.Dense(256, activation='relu'))
        net.add(nn.Dropout(0.5))
        net.add(nn.Dense(120))

    net.initialize(ctx=ctx)
    return net

In [26]:
softmax_cross_entropy = mx.gluon.loss.SoftmaxCrossEntropyLoss()
def accuracy(output, label):
     return nd.mean(output.argmax(axis=1)==label).asscalar()
def evaluate_accuracy(net, data_iter, ctx=mx.cpu()):
    step = len(data_iter)
    acc = 0
    loss = 0
    for data, label in data_iter:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        output_loss = softmax_cross_entropy(output, label)
        acc += accuracy(output, label)
        loss += nd.mean(output_loss).asscalar()
    return loss / step, acc / step

In [27]:
def train(net, data_iter_train, data_iter_val, ctx, 
          epochs=50, lr=0.01, mome=0.9, wd=1e-4, lr_decay=0.5, lr_period=20):

    
    trainer = mx.gluon.Trainer(net.collect_params(),  'sgd', {'learning_rate': lr, 'momentum': mome, 
                                      'wd': wd})
    
    train_loss_list = []
    val_loss_list = []
    
    for epoch in range(epochs):
        train_loss = 0.0
        train_acc = 0.0
        steps = len(data_iter_train)
        if epoch > 0 and epoch % lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for X, y in data_iter_train:

            X, y = X.as_in_context(ctx), y.as_in_context(ctx)

            with mx.autograd.record():
                out = net(X)
                loss = softmax_cross_entropy(out, y)

            loss.backward()
            trainer.step(batch_size)
            train_loss += nd.mean(loss).asscalar()
            train_acc += accuracy(out, y)

        val_loss, val_acc = evaluate_accuracy(net, data_iter_val, ctx)
        train_loss_list.append(train_loss / steps)
        val_loss_list.append(val_loss)
        print("Epoch %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%%" % (
            epoch+1, train_loss/steps, train_acc/steps*100, val_loss, val_acc*100))
        
    return train_loss_list, val_loss_list

In [28]:
net = get_net(ctx)
train_loss_list, val_loss_list = train(net, data_iter_train, data_iter_val, ctx, epochs=100, lr=0.01, \
      mome=0.9, wd=1e-4, lr_decay=0.5, lr_period=20)

Epoch 1. loss: 4.1511, acc: 13.91%, val_loss 2.7966, val_acc 53.20%
Epoch 2. loss: 2.0994, acc: 50.08%, val_loss 1.1548, val_acc 77.45%
Epoch 3. loss: 1.2252, acc: 66.66%, val_loss 0.7859, val_acc 80.58%
Epoch 4. loss: 0.9540, acc: 72.90%, val_loss 0.6525, val_acc 83.42%
Epoch 5. loss: 0.8313, acc: 75.47%, val_loss 0.6044, val_acc 82.24%
Epoch 6. loss: 0.7371, acc: 78.05%, val_loss 0.5689, val_acc 83.42%
Epoch 7. loss: 0.6997, acc: 78.93%, val_loss 0.5356, val_acc 83.08%
Epoch 8. loss: 0.6491, acc: 79.94%, val_loss 0.5264, val_acc 83.42%
Epoch 9. loss: 0.5949, acc: 82.02%, val_loss 0.5186, val_acc 83.51%
Epoch 10. loss: 0.5752, acc: 82.09%, val_loss 0.5088, val_acc 84.05%
Epoch 11. loss: 0.5577, acc: 82.77%, val_loss 0.4907, val_acc 84.49%
Epoch 12. loss: 0.5239, acc: 83.79%, val_loss 0.4845, val_acc 84.15%
Epoch 13. loss: 0.5107, acc: 84.28%, val_loss 0.4849, val_acc 83.90%
Epoch 14. loss: 0.4938, acc: 84.86%, val_loss 0.4709, val_acc 84.84%
Epoch 15. loss: 0.4808, acc: 84.63%, val_lo